## Repères du notebook
    
   1. [Introduction](#introduction)
   
      [Observation des différences de colonnes](#obs)
      
      [Séparation de "location" dans data15](#loc)
      
      [Passage à un seul dataframe](#df)
      
      [Nettoyage: supression des bâtiments résidentiels](#res)
      
      [Nettoyage: doublons identifiants batiments](#double)
      
      
   
   2. [ Traitement valeurs manquantes et aberrantes ](#p1)
   
       [Valeurs manquantes: drop colonnes selon remplissage des variables](#var)
       
       [Nettoyage: forte corrélation des variables explicatives](#corr)
       
       [Nettoyage: intérêt faible des variables pour projet](#bof)
       
       [Nettoyage: individus avec valeurs manquantes et "outliers" pour variables à prédire](#drop)
       
   
   3. [Feature engineering et traitement des données](#p2)
       
      [age des batiments à partir de l'année des données et l'année de construction](#âge)
      
      [Passage au log des targets](#log)
      
      [séparation du dataframe par types de données](#sep)
      
      
   
   4. [Contrôle et export des données nettoyées](#p3)

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from sklearn import decomposition
from sklearn import preprocessing
from sklearn.impute import KNNImputer
import statsmodels.formula.api
import statsmodels.api
import six
import dataframe_image as dfi
import scipy.stats as stats

plt.style.use('seaborn-whitegrid')

#options d'affichage
pd.set_option('display.max_row', 200)
pd.set_option('display.max_column', 70)

## 1. Introduction: Chargement des données et observations <a class="anchor" id="introduction"></a>

In [2]:
data_15 = pd.read_csv('données/building-energy-benchmarking-15.csv')
data_16 = pd.read_csv('données/building-energy-benchmarking-16.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'données/building-energy-benchmarking-15.csv'

In [ ]:
data_15.head()

In [ ]:
data_16.head()

In [ ]:
data_15.info()

In [ ]:
data_16.info()

In [ ]:
print(data_15.columns)
data_15.shape

In [ ]:
data_16.columns

## Observation des différences de colonnes <a class="anchor" id="obs"></a>

In [ ]:
#on cherche les différences de colonnes entre les deux dataframes
mis_col = data_15.columns[~data_15.columns.isin(data_16.columns)]

In [ ]:
mis_col

In [ ]:
#la colonne location de 2015 contient de nombreuses informations séparées dans le dataframe de 2016
data_15['Location']

In [ ]:
data_15['Location'][0]

## Séparation de "location" dans data15 <a class="anchor" id="loc"></a>

In [ ]:
#on utrilise le module abstract syntax trees pour désempacter "location" puis "human address" avec des listes en compréhensions
import ast
data_15['Location'] = [ast.literal_eval(str(item)) for index, item in data_15.Location.iteritems()]
data_15 = pd.concat([data_15.drop(['Location'], axis=1), data_15['Location'].apply(pd.Series)], axis=1)

data_15['human_address'] = [ast.literal_eval(str(item)) for index, item in data_15.human_address.iteritems()]
data_15 = pd.concat([data_15.drop(['human_address'], axis=1), data_15['human_address'].apply(pd.Series)], axis=1)
data_15.head()

In [ ]:
#Renommer pour que les colonnes correspondent:
data_15 = data_15.rename(columns={"latitude":"Latitude", 
                                  "longitude":"Longitude",
                                  "address":"Address", 
                                  "city":"City", 
                                  "state":"State", 
                                  "zip":"ZipCode"})

In [ ]:
#Est-ce qu'elles correspondent aux mêmes informations?
print(data_15['GHGEmissions(MetricTonsCO2e)'].describe())
data_15['GHGEmissionsIntensity(kgCO2e/ft2)'].describe()

In [ ]:
print(data_16['TotalGHGEmissions'].describe())
data_16['GHGEmissionsIntensity'].describe()

## Passage à un seul dataframe<a class="anchor" id="df"></a>

In [ ]:
#Renommer:
data_15 = data_15.rename(columns={'GHGEmissions(MetricTonsCO2e)':'TotalGHGEmissions',
                                  'GHGEmissionsIntensity(kgCO2e/ft2)':'GHGEmissionsIntensity',
                                  'Comment':'Comments'})

In [ ]:
# drop colonnes qui ne sont pas dans les deux dataframe
mis_col = data_15.columns[~data_15.columns.isin(data_16.columns)]
print(mis_col)
data_15.drop(mis_col, axis=1, inplace=True)

In [ ]:
#on vérifie les types avent la concaténation

In [ ]:
dtypes = pd.DataFrame([data_15.dtypes, data_16.dtypes])
dtypes

In [ ]:
#On recast pour que les dtypes correspondent:
data_15[['Latitude','Longitude', 'ZipCode']] = data_15[['Latitude','Longitude', 'ZipCode']].astype('float64')

In [ ]:
data_16.columns[data_16.columns.isin(data_15.columns)]

In [ ]:
#Assembler les deux dataframes:
r_data = pd.concat([data_15[data_16.columns],data_16], axis = 0).sort_values(["DataYear", "OSEBuildingID"])
r_data.shape

In [ ]:
r_data.info()

## Nettoyage: supression des bâtiments résidentiels<a class="anchor" id="res"></a>

In [ ]:
#on s'intéresse au batiments "non-habitations" pour ce projet:
r_data['BuildingType'].unique()


In [ ]:
r_data = r_data[~r_data['BuildingType'].str.contains("Multifamily")]
r_data['BuildingType'].unique()

In [ ]:
r_data.shape

## Nettoyage: doublons identifiants batiments<a class="anchor" id="double"></a>

In [ ]:
#bonne idée reprise ici: se débarasser des batiments en double mais en conservant une moyenne des données continues pour saisir
#une part de l'information: l'évolution entre l'année 2015 et 2016:

mean_columns = ['NumberofBuildings', 'NumberofFloors', 'PropertyGFATotal',
                'PropertyGFAParking', 'PropertyGFABuilding(s)',
                'LargestPropertyUseTypeGFA', 'SecondLargestPropertyUseTypeGFA',
                'ThirdLargestPropertyUseTypeGFA', 'ENERGYSTARScore', 'SiteEUI(kBtu/sf)',
                'SiteEUIWN(kBtu/sf)', 'SourceEUI(kBtu/sf)', 'SourceEUIWN(kBtu/sf)',
                'SiteEnergyUse(kBtu)', 'SiteEnergyUseWN(kBtu)', 'SteamUse(kBtu)',
                'Electricity(kWh)', 'Electricity(kBtu)', 'NaturalGas(therms)',
                'NaturalGas(kBtu)', 'TotalGHGEmissions', 'GHGEmissionsIntensity']
OSEBuilding_means = r_data[['OSEBuildingID']+mean_columns].groupby('OSEBuildingID').mean()
OSEBuilding_means.head()

In [ ]:
#drop les doublons sur l'id des batiments et concaténation des moyennes entre les deux années:
duplicate_building = r_data.drop_duplicates(subset=['OSEBuildingID'], keep='last')
duplicate_building.drop(mean_columns, axis=1, inplace=True)
r_data = pd.merge(duplicate_building, OSEBuilding_means, how='left', on='OSEBuildingID')

In [ ]:
#Vérification pour assurer qu'il n'y ait pas de grosses erreurs:
r_data.info()

## 2. Traitement valeurs manquantes et aberrantes <a class="anchor" id="p1"></a>

## Valeurs manquantes: remplissage des variables<a class="anchor" id="var"></a>

In [ ]:
#nombre total et pourcentage de valeurs manquantes dans le df 
def perc_mv(data):
    Tot_missing_values = data.isna().sum().sum()
    print('total valeurs manquantes: {} '.format(Tot_missing_values))
    Tot_perc_missing_values = (Tot_missing_values/np.product(data.shape))*100
    return Tot_perc_missing_values

In [ ]:
perc_mv(r_data)

In [ ]:
msno.bar(r_data)

In [ ]:
msno.matrix(r_data)

In [ ]:
#Conserver "energy star score" pour dernière étape.
#Conserver uniquement les variables (colonnes) remplies à 80%.
energy_stscore = r_data['ENERGYSTARScore']
r_data = r_data.dropna(thresh = len(r_data)*0.8, axis=1)
msno.bar(r_data)

## Nettoyage: forte corrélation des variables explicatives<a class="anchor" id="corr"></a>

In [ ]:

# Creer correlation matrix
mat_cor = r_data.corr().abs()

# Conserver triangle superieur de correlation matrix
upper = mat_cor.where(np.triu(np.ones(mat_cor.shape), k=1).astype(np.bool))

# trouver les variables avec des corrélations supérieurs à 90%:
to_drop = [column for column in upper.columns if any(upper[column] > 0.90)]
to_drop



In [ ]:
#calculer et observer les corrélations entre les variables
print("matrice des corrélations fortes:")
corr_df = r_data[to_drop].corr(method='pearson')

plt.figure(figsize=(8, 6))
sns.heatmap(corr_df, cmap = 'RdBu_r', annot=True)
plt.show()

In [ ]:
# variables redondantes: drop features pour nos modèles: 
to_drop = ['PropertyGFABuilding(s)',
           'LargestPropertyUseTypeGFA',
           'SiteEUIWN(kBtu/sf)',
           'SourceEUI(kBtu/sf)',
           'SourceEUIWN(kBtu/sf)',
           'Electricity(kWh)']
r_data.drop(to_drop, axis=1, inplace=True)

In [ ]:
r_data.info()

## Nettoyage: intérêt faible des variables pour projet<a class="anchor" id="bof"></a>

In [ ]:
r_data.columns

In [ ]:
#WN: weather normalised, NaturalGas: déja sous forme Kbtu
r_data.drop(['SiteEnergyUseWN(kBtu)','NaturalGas(therms)', 'DefaultData'], axis=1, inplace=True)


In [ ]:
#réintroduction dans dataset d'enrgy star score avant traitement des lignes
r_data['ENERGYSTARScore'] = energy_stscore
r_data.columns

In [ ]:
r_data.describe()

## Nettoyage: individus (lignes) avec valeurs manquantes et outliers pour variables à prédire<a class="anchor" id="drop"></a>

In [ ]:
#drop lignes manquantes dans nos var à prédire
r_data = r_data[~((r_data['SiteEnergyUse(kBtu)'].isnull()) | (r_data['TotalGHGEmissions'].isnull()))]

In [ ]:
#zscore sup à trois écarts types
r_data = r_data[(np.abs(stats.zscore(r_data[['TotalGHGEmissions',
                                             'SiteEnergyUse(kBtu)']] ,
                                        nan_policy='omit')) < 3).all(axis=1)]

## 3. "Feature engineering" et traitement des données<a class="anchor" id="p2"></a>

## age des batiments à partir de l'année des données et l'année de construction<a class="anchor" id="âge"></a>

In [ ]:
#age des batiments plutôt qu'année: feature engineering.
r_data['BuildingAge'] = r_data['DataYear'] - r_data['YearBuilt']
r_data.drop('YearBuilt', axis=1, inplace=True)

In [ ]:
r_data.info()

## Passage au log des targets<a class="anchor" id="log"></a>

In [ ]:
r_data[['SiteEnergyUse(kBtu)']].describe

In [ ]:
r_data[['SiteEnergyUse(kBtu)']].hist(bins=300)

In [ ]:
#On passe au log pour espacer et "normaliser" les données: données et predictions plus lisibles
r_data['SiteEnergyUse(kBtu)'] = np.log(r_data['SiteEnergyUse(kBtu)'])

print(r_data[r_data['SiteEnergyUse(kBtu)'] < 0].head)

r_data.drop(r_data[r_data['SiteEnergyUse(kBtu)'] < 0].index, inplace = True)

r_data[r_data['SiteEnergyUse(kBtu)'] < 0]

In [ ]:
r_data[['SiteEnergyUse(kBtu)']].hist(bins=300)

In [ ]:
#idem pour total emissions carbonne :
r_data[['TotalGHGEmissions']].hist(bins=300)

In [ ]:
r_data['TotalGHGEmissions'] = np.log(r_data['TotalGHGEmissions'])

print(r_data[r_data['TotalGHGEmissions'] < 0].head)

r_data.drop(r_data[r_data['TotalGHGEmissions'] < 0].index, inplace = True)

r_data[r_data['TotalGHGEmissions'] < 0]

In [ ]:
r_data[['TotalGHGEmissions']].hist(bins=300)

## séparation du dataframe par types de données<a class="anchor" id="sep"></a>

In [ ]:
# projet: "L’objectif est de te passer des relevés de consommation annuels"
#séparer données annuels
annual_features = ['SteamUse(kBtu)','Electricity(kBtu)',
                         'NaturalGas(kBtu)','SiteEUI(kBtu/sf)']
r_data_annual = r_data[annual_features]
r_data.drop(annual_features, axis=1, inplace = True)


In [ ]:
#séparer données d'identification
identification_features = ['OSEBuildingID', 'PropertyName', 'Address', 'ZipCode','City', 'State','TaxParcelIdentificationNumber', 'CouncilDistrictCode']
r_data_identification = r_data[identification_features]
r_data.drop(identification_features, axis=1, inplace = True)

## 4. Contrôle et export des données nettoyées<a class="anchor" id="p3"></a>

In [ ]:
#calculer et observer les corrélations entre les variables: vérifier le nettoyage. Les deux variables à prédire 
#sont trés corrélés: pas de pblme pour nos prédictions.
corr = r_data.corr()
mask = np.zeros_like(corr)
mask[np.triu_indices_from(mask)] = True
fig, ax = plt.subplots(figsize=(15,15))
ax = sns.heatmap(corr, annot=True, fmt=".2f", annot_kws={'size':8}, 
                 mask=mask, center=0, cmap="coolwarm")
plt.title(f"Heatmap des corrélations linéaires\n")
plt.show()

In [ ]:
r_data.info()

In [ ]:
msno.bar(r_data)

In [ ]:
perc_mv(r_data)

In [ ]:
#drop informations redondantes:
r_data.drop(['Neighborhood','DataYear'], axis=1, inplace=True)

In [ ]:
num_data = r_data.select_dtypes(include=['int64','float64'])
cat_data = r_data.select_dtypes(exclude=['int64','float64']) 

In [ ]:
cat_data.nunique()

In [ ]:
cat_data.columns

In [ ]:
num_data.columns

In [ ]:
r_data.to_pickle("cleaned_data")